In [1]:
#import dependencies
import pandas as pd
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func, inspect
Base = automap_base()
import requests
from config import gkey
import json
import numpy as np

In [2]:
#create engine and import data from sqlite database
engine = create_engine('sqlite:///../static/data/bredat.sqlite', echo=False)
Base.metadata.create_all(engine)

In [3]:
Base.prepare(engine, reflect=True)

In [4]:
session = Session(engine)

In [5]:
#create dataframe
brew_df = pd.read_sql_query("SELECT * FROM breweries", con=engine)
brew_df

,index,Company Name_x,street,City,State,Zip Code,Zip Plus,Contact First Name,Contact Last Name,Contact Title,...,URL,Year Established_x,latitude,longitude,google_rating,google_numratings,Company Name_y,Employee_y,Annual Sales_y,Year Established_y
0,0,217 Brew Works,217 South St S,Wilson,NC,27893,4911.0,Tom,Curran,Owner,...,217BREWWORKS.COM,2016,35.722939,-77.912148,4.6,83.0,217 Brew Works,14,10453000.0,2016
1,1,All Sevens Brewing,777 Haywood Rd,Asheville,NC,28806,3132.0,None,None,None,...,None,2018,35.578687,-82.594280,4.8,4.0,All Sevens Brewing,23,34511000.0,2018
2,2,Altamont Brewing Co,1050 Haywood Rd,Asheville,NC,28806,2639.0,None,None,None,...,None,2017,35.581549,-82.604400,4.3,169.0,Altamont Brewing Co,23,34511000.0,2017
3,3,Andrews Brewing Co,1026 Main St,Andrews,NC,28901,NaN,None,None,None,...,None,2018,35.200849,-83.824356,4.7,146.0,Andrews Brewing Co,14,17916000.0,2018
4,4,Angry Troll Brewing,222 E Main St # U2,Elkin,NC,28621,3493.0,Cicely,Mc Culloch,Owner,...,ANGRYTROLLBREWING.COM,2015,36.244335,-80.848763,4.3,281.0,Angry Troll Brewing,12,16648000.0,2015
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
275,271,Wise Man Brewing,826 Angelos Bros Ave,Winston Salem,NC,27101,1524.0,Michael D,Beverly,Manager,...,WISEMANBREWING.COM,2017,36.104924,-80.245300,4.7,302.0,Wise Man Brewing,14,5870000.0,2017
276,272,Wooden Robot Brewery LLC,1440 S Tryon St # 110,Charlotte,NC,28203,4263.0,Joshua,Patton,CEO,...,WOODENROBOTBREWERY.COM,2015,35.216754,-80.856676,4.6,530.0,Wooden Robot Brewery LLC,14,7049000.0,2015
277,273,Yesteryears Brewery,300 E Main St # C,Carrboro,NC,27510,2359.0,David,Larsen,Manager,...,None,2016,35.910023,-79.068814,4.6,88.0,Yesteryears Brewery,14,21819000.0,2016
278,274,York Chester Brewing Co,1500 River Dr # 250,Belmont,NC,28012,3587.0,None,None,None,...,YORKCHESTERBREW.COM,2017,35.236705,-81.015616,4.9,56.0,York Chester Brewing Co,14,20830000.0,2017


In [6]:
# load in NC brewery data CSV
nc_commerce_csv = "NC-Breweries_3-24-2020_clean-data.csv"

# make into a data frame
nc_breweries = pd.read_csv(nc_commerce_csv)
nc_breweries

,CompanyName,street,City,State,ZipCode,ZipPlus,ContactFirstName,ContactLastName,ContactTitle,Phone,...,NAICS2,NAICS3,NAICS4,NAICS5,BusinessType,County,Employee,AnnualSales,URL,YearEstablished
0,217 Brew Works,217 South St S,Wilson,NC,27893,4911.0,Tom,Curran,Owner,2529916958,...,NaN,NaN,NaN,NaN,Private,Wilson,14,10453000.0,217BREWWORKS.COM,2016
1,All Sevens Brewing,777 Haywood Rd,Asheville,NC,28806,3132.0,NaN,NaN,NaN,8282259782,...,NaN,NaN,NaN,NaN,Private,Buncombe,23,34511000.0,NaN,2018
2,Altamont Brewing Co,1050 Haywood Rd,Asheville,NC,28806,2639.0,NaN,NaN,NaN,8284123212,...,NaN,NaN,NaN,NaN,Private,Buncombe,23,34511000.0,NaN,2017
3,Andrews Brewing Co,1026 Main St,Andrews,NC,28901,NaN,NaN,NaN,NaN,8283212006,...,NaN,NaN,NaN,NaN,Private,Cherokee,14,17916000.0,NaN,2018
4,Angry Troll Brewing,222 E Main St # U2,Elkin,NC,28621,3493.0,Cicely,Mc Culloch,Owner,3368350103,...,NaN,NaN,NaN,NaN,Private,Surry,12,16648000.0,ANGRYTROLLBREWING.COM,2015
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
908,Woodfin ABC,142 Weaverville Rd,Asheville,NC,28804,1238.0,Tammy,Ballew,Manager,8286589336,...,NaN,NaN,NaN,NaN,Private,Buncombe,5,2240000.0,WOODFINABC.NET,2007
909,Worth the Wine Co,1076 Weddington Woods St NW,Concord,NC,28027,8143.0,Lindsay G,Hitt,Owner,7040001111,...,NaN,NaN,NaN,NaN,Private,Cabarrus,2,465000.0,NaN,2016
910,Ws Wine Market LLC,1231 Creekshire Way,Winston Salem,NC,27103,3057.0,Elizabeth A,Binder,Manager,3368936950,...,NaN,NaN,NaN,NaN,Private,Forsyth,1,482000.0,NaN,2017
911,Youngsville ABC Store,101 N College St,Youngsville,NC,27596,NaN,Laurie,Leedy,Manager,9195563590,...,NaN,NaN,NaN,NaN,Private,Franklin,3,654000.0,NaN,2002


In [7]:
nc_breweries_tomerge = nc_breweries[['CompanyName','street','Employee','AnnualSales','YearEstablished']]
nc_breweries_tomerge

,CompanyName,street,Employee,AnnualSales,YearEstablished
0,217 Brew Works,217 South St S,14,10453000.0,2016
1,All Sevens Brewing,777 Haywood Rd,23,34511000.0,2018
2,Altamont Brewing Co,1050 Haywood Rd,23,34511000.0,2017
3,Andrews Brewing Co,1026 Main St,14,17916000.0,2018
4,Angry Troll Brewing,222 E Main St # U2,12,16648000.0,2015
...,...,...,...,...,...
908,Woodfin ABC,142 Weaverville Rd,5,2240000.0,2007
909,Worth the Wine Co,1076 Weddington Woods St NW,2,465000.0,2016
910,Ws Wine Market LLC,1231 Creekshire Way,1,482000.0,2017
911,Youngsville ABC Store,101 N College St,3,654000.0,2002


In [8]:
# merge datasets
brew_merge = pd.merge(brew_df, nc_breweries_tomerge, on="street", how="left")
brew_merge

,index,Company Name_x,street,City,State,Zip Code,Zip Plus,Contact First Name,Contact Last Name,Contact Title,...,google_rating,google_numratings,Company Name_y,Employee_y,Annual Sales_y,Year Established_y,CompanyName,Employee,AnnualSales,YearEstablished
0,0,217 Brew Works,217 South St S,Wilson,NC,27893,4911.0,Tom,Curran,Owner,...,4.6,83.0,217 Brew Works,14,10453000.0,2016,217 Brew Works,14,10453000.0,2016
1,1,All Sevens Brewing,777 Haywood Rd,Asheville,NC,28806,3132.0,None,None,None,...,4.8,4.0,All Sevens Brewing,23,34511000.0,2018,All Sevens Brewing,23,34511000.0,2018
2,2,Altamont Brewing Co,1050 Haywood Rd,Asheville,NC,28806,2639.0,None,None,None,...,4.3,169.0,Altamont Brewing Co,23,34511000.0,2017,Altamont Brewing Co,23,34511000.0,2017
3,3,Andrews Brewing Co,1026 Main St,Andrews,NC,28901,NaN,None,None,None,...,4.7,146.0,Andrews Brewing Co,14,17916000.0,2018,Andrews Brewing Co,14,17916000.0,2018
4,4,Angry Troll Brewing,222 E Main St # U2,Elkin,NC,28621,3493.0,Cicely,Mc Culloch,Owner,...,4.3,281.0,Angry Troll Brewing,12,16648000.0,2015,Angry Troll Brewing,12,16648000.0,2015
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
283,271,Wise Man Brewing,826 Angelos Bros Ave,Winston Salem,NC,27101,1524.0,Michael D,Beverly,Manager,...,4.7,302.0,Wise Man Brewing,14,5870000.0,2017,Wise Man Brewing,14,5870000.0,2017
284,272,Wooden Robot Brewery LLC,1440 S Tryon St # 110,Charlotte,NC,28203,4263.0,Joshua,Patton,CEO,...,4.6,530.0,Wooden Robot Brewery LLC,14,7049000.0,2015,Wooden Robot Brewery LLC,14,7049000.0,2015
285,273,Yesteryears Brewery,300 E Main St # C,Carrboro,NC,27510,2359.0,David,Larsen,Manager,...,4.6,88.0,Yesteryears Brewery,14,21819000.0,2016,Yesteryears Brewery,14,21819000.0,2016
286,274,York Chester Brewing Co,1500 River Dr # 250,Belmont,NC,28012,3587.0,None,None,None,...,4.9,56.0,York Chester Brewing Co,14,20830000.0,2017,York Chester Brewing Co,14,20830000.0,2017


In [9]:
# add data to sqlite database
brew_merge.to_sql('breweries', con=engine, if_exists='replace', index=False)

In [10]:
#test sqlite data
engine.execute("SELECT * FROM breweries").fetchall()[0]

(0, '217 Brew Works', '217 South St S', 'Wilson', 'NC', 27893, 4911.0, 'Tom', 'Curran', 'Owner', 2529916958, 'Brewers (mfrs)', 312120, None, None, None, None, 'Private', 'Wilson', 14, 10453000.0, '217BREWWORKS.COM', 2016, 35.7229387, -77.912148, 4.6, 83.0, '217 Brew Works', 14, 10453000.0, 2016, '217 Brew Works', 14, 10453000.0, 2016)

In [12]:
# create sqlite database of naics data
nc_breweries_database = nc_breweries[nc_breweries['PrimaryNAICS'] == 312120]
# add data to sqlite database
nc_breweries_database.to_sql('business', con=engine, if_exists='replace', index=True)

In [13]:
#test sqlite data
engine.execute("SELECT * FROM business").fetchall()[0]

(0, '217 Brew Works', '217 South St S', 'Wilson', 'NC', 27893, 4911.0, 'Tom', 'Curran', 'Owner', 2529916958, 'Brewers (mfrs)', 312120, None, None, None, None, 'Private', 'Wilson', 14, 10453000.0, '217BREWWORKS.COM', 2016)